# Lighthill's Elongated-Body Theory (EBT)

Lighthill's large-amplitude elongated-body theory utilizes reactive (rather than resistive) theory in order to interpret observational data on animal movements of arbitrary large amplitude. Reactive theory emphasizes reactive forces between a small volume of water and the parts of the animal's surface in contact with it. 

Lighthill's large-amplitude elongated-body theory applies to fish that swim with carangiform motion. Carangiform motion is characterized by fast forward motion, larger wave amplitudes towards the tail in the last half or one-third of the body length than in the anterior, and slower phase of lateral oscillations in this posterior section than that of the anterior. As a result, reactive forces seem to dominate in carangiform motion because the acceleration of water passed by the fish takes place very fast when the posterior section passes and flicks it into motion through the instantaneously acting virtual-mass effect.

The goal of Lighthill's 1971 paper is to extend reactive theory and slender-body theory so that it can be applied to interpret observational data on animal movements of arbitrary large amplitude. In this model, it is hopefully applied to carangiform movements. 

In terms of force and propulsion, Lighthill's elongated-body theory makes use of three principles in a frame of reference in which the water far from the fish is at rest:

1. Water momentum near a section of fish is in a direction perpendicular to the backbone and has magnitude equal to the virtual mass, $m$ per unit length, times the component $w$ of fish velocity in that direction.
2. Thrust can be obtained by considering rate of change of momentum within a volume enclosing the fish whose boundary at each instant includes a flat surface $\Pi$ perpendicular to the caudal fin through its posterior end.
3. In the momentum balance it is necessary to take into account transfer of momentum across $\Pi$ not only by convection but also by the action of the resultant $\frac{1}{2} mw^2$ of the pressures generated by the motions within the plane $\Pi$.

The coordinates used by Lighthill are:

* $y$-axis as the vertical
* $x$-axis and $z$-axis as the horizontal
* Lagrangian coordinate a along the fish body that takes values from 0 (tail) to $l$ (head), the length of the fish

where a left-handed system of axes is used.

The fish swims at a fixed horizontal level ($y=0$), where the constant mean direction of swimming is taken as that of the positive $x$-axis. Additionally, the position coordinates are regarded as $(x(a,t), z(a,t))$ as the point moves due (i) to swimming actions, (ii) to the resulting forward motion of the fish, and (iii) to any lateral recoil motions that may be involuntarily produced.

In addition, we have several notation:

* $(u,w)$ is the horizontal velocity vector (in the Lagrangian fish body frame), where $u$ is the tangential component (forward direction) and $w$ is the perpendicular component (lateral direction) of the fish body
* $(P,Q)$ is the force vector where $P$ is thrust and $Q$ is sideforce; $P$ is forward direction, $Q$ is lateral direction
* $V$ is the magnitude of the $(u,w)$
* $W$ is the component perpendicular to the direction of mean motion

In an elongated body form, the virtual mass is large in respect of the $w$ motions, while the $u$ motions have negligible virtual mass.

The inextensibility of the fish's spinal column requires $ \left(\frac{\partial x}{\partial a}\right)^2 + \left(\frac{\partial z}{\partial a}\right)^2 = 1$. 

The horizontal velocity vector (in a left-handed $x-z$ coordinate system) $\left(\frac{\partial x}{\partial t}, \frac{\partial z}{\partial t} \right)$ has a tangential component (relative to the spinal column) $u= \frac{\partial x}{\partial t} \frac{\partial x}{\partial a} + \frac{\partial z}{\partial t} \frac{\partial z}{\partial a}$ and a perpendicular component $w= \frac{\partial z}{\partial t} \frac{\partial x}{\partial a} - \frac{\partial x}{\partial t} \frac{\partial z}{\partial a}$.

The momentum per unit length of fish is represented by the vector $mw\left(-\frac{\partial z}{\partial a}, \frac{\partial x}{\partial a} \right)$ where the factor in parentheses is a unit vector in the $w$-direction. $m$ is the given cross-section of the fish, which can be approximated by $\frac{1}{4} \pi \rho s^2$, where $\rho$ is the water density and $s$ is the depth of the cross-section. $m$ should be a fixed, known function $m(a)$.

It is difficult to use the rate of change of the momentum in the wake. A better approach would be to study the rate of change of momentum in a part $V$ that excludes the wake. This is done by considering a vertical plane $\Pi$ intersecting the tip of the caudal fin at right angles to the tip of the spinal column. This plane swings around as the fine moves. At each instant, the part $V$ of water whose momentum is considered is that half-space, bounded by the plane $\Pi$, which includes the fish and excludes the wake. The momentum in $V$ can be written as the integral of $mw \left( -\frac{\partial z}{\partial a}, \frac{\partial x}{\partial a} \right)$ from $0$ to $l$ with respect to $a$, and its rate of change can be written as the sum of three terms:

1. rate of change due to convection of momentum out of $V$ across the plane $\Pi$
2. rate of change due to the pressure force acting across $\Pi$
3. minus the reactive force $(P,Q)$ with which the fluid acts on the fish

In general, 
$(P,Q) = \left[mw \left( \frac{\partial z}{\partial t}, -\frac{\partial x}{\partial t} \right) - \frac{1}{2} mw^2 \left(\frac{\partial x}{\partial a}, \frac{\partial z}{\partial a} \right) \right]_{a=0} - \frac{d}{dt} \int_{0}^{l} mw \left(-\frac{\partial z}{\partial a}, \frac{\partial x}{\partial a} \right) da$

The mean thrust can be written as $\overline{P} = \overline{ \left[ mw \left(\frac{\partial z}{\partial t}-\frac{1}{2} w \frac{\partial x}{\partial a} \right) \right]_{a=0}}$


## Symbolic Implementation

In the first attempt, we will implement Lighthill's model symbolically. This means that our functions must be analytically solveable; reasonable examples include sinusoidal functions.

We will mainly be working with the SymPy package, which can handle symbolic differentiation and integration.

In [4]:
%matplotlib inline

from scipy import spatial
import math
import matplotlib.pyplot as plt
import numpy as np
from sympy import *

a, t = symbols('a t')
m0, w0, omega, z0 = symbols('m0 w0 omega z0')

In [254]:
""" 
Important variables:
P: thrust
Q: sideforce
m: the virtual mass per unit length
w: the perpendicular component of the fish body's velocity, relative to the body
z: the Cartesian coordinate for side motion
x: the Cartesian coordinate for forward motion
a: the Lagrangian coordinate along the fish body, measured from the caudal fin
u: the tangential component of the fish body's velocity
t: time
"""

"""
Pcomponent(m,w,x,z,l) should output the P component of our reactive force.
Similarly, Qcomponent(m,w,x,z,l) should output the Q component.
"""
def Pcomponent(m,w,x,z,l):

    P = m*w*diff(z,t)-(1/2)*m*w**2*diff(x,a) - diff(integrate(
        m*w*(-diff(z,a)),(a,0,l)) ,t)
#     print(P)
#     print(P.subs(a,0))
    return P.subs(a,0)

def Qcomponent(m,w,x,z,l):
    Q = -(m*w*diff(x,t)+(1/2)*m*w**2*diff(z,a)) - diff(integrate(
        m*w*diff(x,a),(a,0,l)),t)
#     print(Q)
#     print(Q.subs(a,0))
    return Q.subs(a,0)

In [255]:
"""
reactiveForce(m,w,x,z,l) should output the force vector based on 
the P component and Q component defined above.
"""
def reactiveForce(m,w,x,z,l):
    reactiveForceVector = np.array([Pcomponent(
        m,w,x,z,l),Qcomponent(m,w,x,z,l)])
    return reactiveForceVector

In [256]:
"""
averageThrust(m,w,x,z,l) should output the average thrust based on Pcomponent.
"""

def averageThrust(m,w,x,z,l):
    Pavg = m*w*diff(z,t)-(1/2)*m*w**2*diff(x,a)
    return Pavg.subs(a,0)

Tests for Pcomponent, Qcomponent, reactiveForce, and averageThrust below:

In [257]:
Pcomponent(m0,w0*cos(omega*t),0,z0*sin(omega*t),1)

m0*omega*w0*z0*cos(omega*t)**2

In [258]:
Qcomponent(m0,w0*cos(omega*t),0,z0*sin(omega*t),1)

0

In [262]:
reactiveForce(m0,w0*cos(omega*t),0,z0*sin(omega*t),1)

array([m0*omega*w0*z0*cos(omega*t)**2, 0], dtype=object)

In [261]:
# should probably be the same as the instantaneous value of Pcomponent
averageThrust(m0,w0*cos(omega*t),0,z0*sin(omega*t),1)

m0*omega*w0*z0*cos(omega*t)**2

We will attempt to plot some waveforms as well as extract steps such as dx, dz, and da. Our overall goal is to be able to see a simplistic waveform model (best through animation), test for consistency in coordinate axes, and utilize our results for force production.

In [105]:
"""Here, we want to try to make an array with certain steps (dx, dz, da) so that I can
later retrieve each point for analysis. We will be forming a simple sinusoidal wave.
In our case, we have 50 points from 0-1 so dx = 1/49

Next goal: animate plot, feed in kinematic data, clean up code in terms of 
renaming variables such as step size
"""
class Lighthill:
    
    
    """
    Initializer for the Lighthill class. 
    t: an array of time values at which data was taken.
    x: an array of x values. Ideally, x would range from 0 to 1.
    z: an array of z values. z should also range from 0 to 1.
    m: a float that represents the virtual mass.
    
    Arrays should be same number of elements.
    """
    def __init__(self, t, x, z, m):
        self.t = t
        self.x = x
        self.z = z
        
        self.body = np.array(list(zip(x,z)))
        
        self.dt = np.fromiter([t[i+1]-t[i] for i in range(len(t)-1)],float)
        self.dx = np.fromiter([x[i+1]-x[i] for i in range(len(x)-1)],float)
        self.dz = np.fromiter([z[i+1]-z[i] for i in range(len(z)-1)],float)
        self.da = np.fromiter([sqrt((x[i]-x[i+1])**2+(z[i]-z[i+1])**2) for i in range(len(x)-1)],float)
        
        self.m = 1.2 # approx 1.2 as starting point
        self.w = (self.dz/self.dt)*(self.dx/self.da)-(self.dx/self.dt)*(self.dz/self.da)
    
        self.thrustForce = (self.m*self.w*(self.dz/self.dt)-(1/2)*self.m*self.w**2*(self.dx/self.da)
                            -np.sum(self.m*self.w*(-self.dz))/self.dt)
        
        self.sideForce = (self.m*self.w*(-self.dx/self.dt)-(1/2)*self.m*self.w**2*(self.dz/self.da)
                          -np.sum(self.m*self.w*self.dx)/self.dt)
                
        self.forceVector = np.array(list(zip(self.thrustForce,self.sideForce)))
        
        self.inextBool = None
   
    
#     def force(self):
#         # to calculate thrust force at a point
#         # thrustSumTerm = 
#         self.thrustForce = self.m*self.w*(self.dz/self.dt)-(1/2)*self.m*self.w**2*(self.dx/self.da) /
#         -np.sum(self.m*self.w*(-self.dz))/self.dt

#         # sideSumTerm = 
#         self.sideForce = self.m*self.w*(-self.dx/self.dt)-(1/2)*self.m*self.w**2*(self.dz/self.da) /
#         np.sum(self.m*self.w*self.dx)-/self.dt

#         self.forceVector = np.array(list(zip(self.thrustForce,self.sideForce)))
#         print("Force Vector (P,Q): ", self.forceVector)
        
    def waveform(self):
        plt.plot(self.thrustForce,self.sideForce)
        #return dz/da
        
        
    """
    inextensibilityCheck checks the inextensibility of the fish body. It returns True if all of the elements in the
    array return 1 and returns False otherwise. inextensibilityCheck takes in an argument err whose default value is
    0. It can be changed to a float, which allows for +/- tolerance at which the function still returns True.
    """
    def inextensibilityCheck(self,err=0):
        
        inextensibilityArray = (self.dx/self.da)**2 + (self.dz/self.da)**2
        print(inextensibilityArray)
        self.inextBool = all(i>=(1-err) and i<=(1+err) for i in inextensibilityArray)
        

In [57]:
tempList = [1, 1., 1.05, 1.02, 1
           ]
all(i>=.95 and i<=1.05 for i in tempList)

True

In [106]:
tTemp = np.linspace(0,1,10,endpoint=True)
xTemp = np.linspace(0,1,10,endpoint=True)
# np.empty(stepSize-1); dx.fill(1/(stepSize-1)) for uniform dx
zTemp = np.sin(xTemp) # approx 1 or 0.5 

fishy = Lighthill(tTemp, xTemp, zTemp, 1.2)
# print(fishy.t)
# print(fishy.z)
# print(fishy.x)
# print(fishy.dt)
# print(fishy.dz)
# print(fishy.dx)
# print(fishy.da)
# fishy.waveform()
print(fishy.forceVector)
fishy.inextensibilityCheck()
# should return True
fishy.inextBool

[[ 1.28053911e-16 -1.33226763e-16]
 [ 1.28053911e-16 -1.33226763e-16]
 [ 2.56107823e-16 -2.66453526e-16]
 [ 1.28053911e-16 -1.33226763e-16]
 [ 1.28053911e-16 -1.33226763e-16]
 [ 1.28053911e-16 -1.33226763e-16]
 [ 1.28053911e-16 -1.33226763e-16]
 [ 1.28053911e-16 -1.33226763e-16]
 [ 1.28053911e-16 -1.33226763e-16]]
[1. 1. 1. 1. 1. 1. 1. 1. 1.]


False

## Numerical Implementation

Once our symbolic model works, we will implement the model numerically. This implementation will be the most useful when we have arbitrary data and functions that cannot be solved analytically. 

We will mainly be working with numPy, which can handle numerical differentiation and integration.